## Cats vs Dogs classification with a small dataset

We are going to use the Kaggle competition [Cats vs Dogs](https://www.kaggle.com/c/dogs-vs-cats/data) dataset with over 25k labelled images in the train set. We are going to use only a small part of the dataset for our purposes:

- 2k images for training (1k cats + 1k dogs)
- 2k images for validation (model selection)
- 2k images for testing

The prepared dataset temporarily is available [here](http://rs1.sze.hu/~katihi/titkos/cats_vs_dogs.zip).

See a related article: Elson, J., Douceur, J. R., Howell, J., & Saul, J. (2007). Asirra: a CAPTCHA that exploits interest-aligned manual image categorization

In [5]:
from keras.initializers import he_normal
from keras.regularizers import L2
from keras.optimizers import SGD, Adam, RMSprop, AdamW
from sklearn.metrics import classification_report, confusion_matrix
from IPython.display import Image
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from keras.datasets import mnist, cifar10
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Activation, AveragePooling2D, Conv2D, Add 
from keras.layers import MaxPooling2D, Reshape, Dropout, BatchNormalization, GlobalAveragePooling2D
from keras.utils import to_categorical, plot_model, set_random_seed

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline

In [6]:
if 0:
    import requests
    import zipfile
    import io
    
    url = "http://rs1.sze.hu/~katihi/titkos/cats_vs_dogs.zip"
    
    response = requests.get(url)
    
    if response.status_code == 200:
        with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
            zip_ref.extractall(".") 
        print("File downloaded and extracted successfully!")
    else:
        print(f"Failed to download file. Status code: {response.status_code}")

### Let's build some simple models for baseline

Since we are working with such a small number of images, we definitely need the ImageDataGenerator modifications for the images.

With the `flow_from_directory` function we can specify:
- the directory we want to read the data from
- `target_size`: the shape we want to resize to
- `batch_size`

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def create_flows(width, height, batch_size=32):
    #VGG input shape is: 224*224, InceptionResNetV2 input shape is 299*299

    train_dir = "cats_vs_dogs/train/"
    val_dir =   "cats_vs_dogs/validation/"
    test_dir =  "cats_vs_dogs/test/"
    
    train_datagen = ImageDataGenerator(rescale=1/255,
                   width_shift_range=0.1,
                   height_shift_range=0.1,
                   horizontal_flip=True)

    
    return train_flow, val_flow, test_flow

In [8]:
width, height, batch_size = 64, 64, 32



## Simple CNN with MaxPooling/GlobalAveragePooling

Let's build a very simple CNN model for binary classification (we have two classes) with

- 2 consecutive Conv2D layers
- (1 MaxPooling layer + a flattening layer) or (1 GlobalAveragePooling)
- 2 fully connected Dense layers

# Reduce model complexity

## Transfer learning

We are going to use an already pre-trained model with its weights and just change the final dense/prediction layers.
Available pre-trained models in Keras can be found [here](https://keras.io/api/applications/)

These models were trained on millions of images and 1000 classes on the ImageNet challenge dataset. Those classes contain several cat and dog breeds.

In [9]:
from keras.applications.vgg16 import VGG16


 - We have to change the prediction layer to two classes
 - We need to freeze most of the layers we don't want re-train the whole model, and only unfreeze the final few layers.

- We freeze the VGG layers (set their weights non-trainable)
- cut the prediction layer
- replace it with our Dense layers

# Preprocessing input

- We should use the same preprocessing as it was in the original training pipeline

In [10]:
from keras.applications.vgg16 import preprocess_input

def create_flows_with_preprocess(width, height, batch_size=32):
    #VGG input shape is: 224*224, InceptionResNetV2 input shape is 299*299


    return train_flow, val_flow, test_flow

### InceptionResNetV2

Let's try to fine-tune an even better and deeper model, the [`InceptionResNetV2`](https://arxiv.org/abs/1602.07261)

In [11]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input

inception = InceptionResNetV2(include_top=False, input_shape=(299, 299, 3))


## Let's look at the model's predictions and its mistakes

In [12]:
from keras.preprocessing.image import load_img, img_to_array

def predict(filename):
    img = load_img(filename, target_size=(299, 299, 3))
    img = img_to_array(img).reshape(1, 299, 299, 3) / 255

    return model.predict(img)[0][0]

### Dogs that were incorrectly identified as cats

In [13]:
for idx in range(2000, 3000):
    filename = f"cats_vs_dogs/test/dog/dog.{idx}.jpg"


### Cats that were incorrectly identified as dogs

In [14]:
for idx in range(2000, 3000):
    filename = f"cats_vs_dogs/test/cat/cat.{idx}.jpg"
